In [14]:
"""
@author: pho
"""
## Automatic debugger on exception raised
# %pdb on
%pdb off
%load_ext autoreload
%autoreload 2
import sys
import importlib
from pathlib import Path
from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd

%config Completer.use_jedi = False

# # required to enable non-blocking interaction:
%gui qt5

import os
import numpy as np
import pandas as pd
from pathlib import Path
# import pickle
import dill as pickle

from neuropy.utils.load_exported import LoadXml, find_session_xml # for compute_with_params_loaded_from_xml

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
pwd()

In [2]:
%pdb on

Automatic pdb calling has been turned ON


In [3]:
# cd(r'C:\\Users\\pho\\repos\\cnn-ripple')

from src.cnn.PhoRippleDetectionTesting import ExtendedRippleDetection, main_compute_with_params_loaded_from_xml

local_session_parent_path = Path(r'W:\Data\KDIBA\gor01\one')
local_session_names_list = ['2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_1-22-43', '2006-6-09_3-23-37', '2006-6-12_15-55-31', '2006-6-13_14-42-6']
local_session_paths_list = [local_session_parent_path.joinpath(a_name).resolve() for a_name in local_session_names_list]

local_session_path: Path = local_session_paths_list[0]
local_session_path

WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-07_11-26-53')

In [6]:
session_xml_filepath, session_stem, local_session_path = find_session_xml(local_session_path)
out_xml_dict, d = LoadXml(session_xml_filepath)

active_shank_channels_lists = out_xml_dict['AnatGrps']
active_shank_channels_lists = [active_shank_channels_lists[i] for i in [0,1,2]]
print(f"active_shank_channels_lists: {active_shank_channels_lists}")

active_shank_channels_lists: [[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15], [16, 17, 18, 19, 20, 21, 22, 23]]


active_shank_channels_lists: [[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15], [16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31], [32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46, 47], [48, 49, 50, 51, 52, 53, 54, 55], [56, 57, 58, 59, 60, 61, 62, 63], [64, 65, 66, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 77, 78, 79], [80, 81, 82, 83, 84, 85, 86, 87], [88, 89, 90, 91, 92, 93, 94, 95]]


In [8]:
## Build the detector:
test_detector = ExtendedRippleDetection(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
ripple_df, out_all_ripple_results = test_detector.compute(**({'active_session_folder': local_session_path,
     'numchannel': out_xml_dict['nChannels'], 'srLfp': out_xml_dict['lfpSampleRate'], 'active_shank_channels_lists': active_shank_channels_lists,
     'overlapping': True, 'window_size': 0.0128, 'window_stride': 0.0064}))

# out_all_ripple_results
ripple_df.to_pickle(local_session_path.joinpath('ripple_df.pkl'))
print(f'done. Exiting.')

Loading CNN model... WARNING:tensorflow:SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.
Done!
active_session_folder: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53
active_session_eeg_data_filepath: W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.eeg
Shape of loaded data:  (2476505, 96)
Downsampling data from 1252 Hz to 1250 Hz... Done!
working on shank 0 with channels: [0, 1, 2, 3, 4, 5, 6, 7]...
Normalizing data... Done!
Shape of loaded data after downsampling and z-score:  (2472549, 8)
Generating windows... Done!
9659/9659 [==============================] - 37s 4ms/step
Done!
Getting detected ripples indexes and times... Done!
working on shank 1 with channels: [8, 9, 10, 11, 12, 13, 14, 15]...
Normalizing data... Done!
Shape of loaded data afte

In [22]:
test_detector.__dir__()

['active_session_folder',
 'active_session_eeg_data_filepath',
 'loaded_eeg_data',
 'out_all_ripple_results',
 'ripple_df',
 'optimizer',
 'model',
 '__module__',
 '__doc__',
 '__init__',
 'compute',
 'predicted_ripples_dataframe_csv_save_filepath',
 'object_save_filepath',
 'ripple_dataframe_pickle_save_filepath',
 '__getstate__',
 '__setstate__',
 'save',
 'readmulti',
 '_downsample_data',
 '_z_score_normalization',
 '_run_single_shank_computation',
 'load_eeg_data',
 'compute_ripples',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [21]:
test_detector.model # keras.engine.sequential.Sequential
test_detector.optimizer # keras.optimizers.optimizer_v2.adam.Adam

In [11]:
test_detector.active_session_folder

WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-07_11-26-53')

In [15]:
# Test pickling the whole ExtendedRippleDetection object
out_ripple_detector_filepath = test_detector.active_session_folder.joinpath('ripple_detector.pkl')
with open(out_ripple_detector_filepath, 'wb') as f:
    print(f'saving entire ripple detector object to {str(out_ripple_detector_filepath)}...')
    pickle.dump(test_detector, f)
print(f'done.')

In [29]:
## Unpickle the object from file:
in_ripple_detector_filepath = out_ripple_detector_filepath
with open(in_ripple_detector_filepath, 'rb') as f:
    print(f'loading pickled ripple detector object from {str(in_ripple_detector_filepath)}...')
    loaded_ripple_detector = pickle.load(f)
print(f'done.')
loaded_ripple_detector

In [31]:
in_ripple_detector_filepath = Path(r'W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\ripple_detector.pkl')
loaded_ripple_detector = ExtendedRippleDetection.load(in_ripple_detector_filepath)
loaded_ripple_detector

loading pickled ripple detector object from W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\ripple_detector.pkl...
Loading CNN model... WARNING:tensorflow:SavedModel saved prior to TF 2.5 detected when loading Keras model. Please ensure that you are saving the model with model.save() or tf.keras.models.save_model(), *NOT* tf.saved_model.save(). To confirm, there should be a file named "keras_metadata.pb" in the SavedModel directory.


Done!
done.


In [ ]:
test_detector, ripple_df, out_all_ripple_results, out_all_ripple_results = main_compute_with_params_loaded_from_xml(active_local_session_path)

In [9]:

out_all_ripple_results['preprocessed_data']['data']


array([[ -991., -1215., -1196., ...,  1432.,  2138.,  2170.],
       [-1049., -1303., -1316., ...,  1240.,  1987.,  2030.],
       [-1069., -1341., -1337., ...,  1338.,  1952.,  1962.],
       ...,
       [  268.,    33.,  -107., ...,  -694.,  -206.,  -325.],
       [  272.,   107.,   -92., ...,  -196.,   333.,    47.],
       [   91.,     5.,  -161., ...,   152.,   695.,   502.]],
      dtype=float16)

In [24]:
out_all_ripple_results['computation_params']

In [26]:
print(list(out_all_ripple_results['computation_params'].keys()))

['overlapping', 'window_size', 'stride', 'threshold', 'learning_rate', 'beta_1', 'beta_2', 'epsilon', 'amsgrad']


In [27]:
from neuropy.utils.dynamic_container import DynamicContainer, override_dict, overriding_dict_with, get_dict_subset

# ['overlapping', 'window_size', 'stride', 'threshold', 'learning_rate', 'beta_1', 'beta_2', 'epsilon', 'amsgrad']
get_dict_subset(out_all_ripple_results['computation_params'], ['learning_rate', 'beta_1', 'beta_2', 'epsilon', 'amsgrad'])

{'learning_rate': 0.001,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [28]:
test_detector.save()

saving entire ripple detector object to W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\ripple_detector.pkl...
done.
